In [1]:
# loads the model built from here and evaluates faces
# https://github.com/WuJie1010/Facial-Expression-Recognition.Pytorch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.autograd import Variable

from PIL import Image
from skimage import io
from skimage.transform import resize as rs

cut_size = 44



In [2]:
%run "./transforms"

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 7)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = F.dropout(out, p=0.5, training=self.training)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [4]:
model = '/dbfs/mnt/bs/Models/PrivateTest_model.t7'
net = VGG('VGG19')
checkpoint = torch.load(model, map_location='cpu')

net.load_state_dict(checkpoint['net'])
net.eval()  

In [5]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
  
transform_test = Compose([
    TenCrop(cut_size),
    Lambda(lambda crops: torch.stack([ToTensor()(crop) for crop in crops])),
])
  
def eval_pyTorch_emotion(image):
  # prep image
  gray = rgb2gray(image)
  im_pil = Image.fromarray(gray)
  gray = rs(gray, (48,48)).astype(np.uint8)
  img = gray[:, :, np.newaxis]
  img = np.concatenate((img, img, img), axis=2)
  img = Image.fromarray(img)
  inputs = transform_test(img)
  class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
  ncrops, c, h, w = np.shape(inputs)
  inputs = inputs.view(-1, c, h, w)
  inputs = Variable(inputs, volatile=True)
  outputs = net(inputs)
  outputs_avg = outputs.view(ncrops, -1).mean(0)  # avg over crops
  score = F.softmax(outputs_avg)
  _, predicted = torch.max(outputs_avg.data, 0)
  
  #print (score)
  #print (predicted)
  #print("The Expression is %s" %str(class_names[int(predicted.cpu().numpy())]))
  return score
  